In [9]:
import numpy as np
winSize = 500
def matPic (line):
  cols = line.split(',')
  counts = [int(i) for i in cols]
  import math
  import numpy as np
  pic = np.zeros((100, 100), float)
  if sum(counts) == 0:
    return pic
  binSize = math.floor(winSize/100)
  binTotal = math.floor(winSize/binSize)
  binned = []
  #Binning
  for i in range (0,binTotal):
    position = i*binSize
    if i == binTotal-1:
      binned.append(np.sum(counts[position:len(counts)]))
    else:
      binned.append(np.sum(counts[position:position+binSize]))

  binned = (99*(binned - np.min(binned))/np.ptp(binned)).astype(int) 
  for i in range(0,len(binned)):
    pic[binned[i],i] = 1
  
  return pic

In [11]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import tensorflow

# Num rows
num_rows = 253897
batch_size = 500

def generate_arrays_from_file(path, batchsize):
    inputs = []
    targets = []
    batchcount = 0
    while True:
        with open(path) as f:
            for line in f:
                x,y = line.split('\t')
                inputs.append(matPic(x))
                targets.append(y)
                batchcount += 1
                if batchcount > batchsize:
                  X = np.array(inputs, dtype='int').reshape(-1,100,100)
                  y = np.array(targets, dtype='int')
                  yield (X, y)
                  inputs = []
                  targets = []
                  batchcount = 0

In [13]:
#TensorFlow and tf.keras
import tensorflow as tf
from tensorflow import keras


model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(2,64, padding='same',
                           input_shape=(100,100)),
    tf.keras.layers.AveragePooling1D(2, strides=2),
    tf.keras.layers.Conv1D(2,32, padding='same'),
    tf.keras.layers.Conv1D(2,32, padding='same'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
model.fit(generate_arrays_from_file("drive/MyDrive/ResearchMahony2023/Allo/narrow_train_new.counts", batch_size),
                    steps_per_epoch=num_rows / batch_size, epochs=3)

Epoch 1/3
507/507 [==============================] - 255s 480ms/step - loss: 0.1748 - accuracy: 0.9363
Epoch 2/3
507/507 [==============================] - 244s 480ms/step - loss: 0.1562 - accuracy: 0.9425
Epoch 3/3
507/507 [==============================] - 240s 473ms/step - loss: 0.1540 - accuracy: 0.9435


In [14]:
model(np.zeros((100, 100), float).reshape(-1,100,100), training=False).numpy()[0][0]

0.011889636

In [15]:
model_json = model.to_json()
with open("drive/MyDrive/ResearchMahony2023/narrow.json", "w") as json_file:
    json_file.write(model_json)
    model.save_weights("drive/MyDrive/ResearchMahony2023/narrow.h5")
print("Saved model to disk")

Saved model to disk


In [16]:
print(tf.__version__)

2.11.0
